In [7]:
import os
import torch
import torch.functional as F
import torchvision
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from loader import General_Loader_withpath, Loader, Loader2, General_Loader
import matplotlib.pyplot as plt
from models.resnet_128 import *

In [44]:
os.environ["CUDA_VISIBLE_DEVICES"]='6'
server_name = 'hinton'
parameter_path = f'/home/{server_name}/NAS_AIlab_dataset/personal/heo_yunjae/Parameters/Uncertainty/h2lal/cifar10/h2l'
data_path = f'/home/{server_name}/NAS_AIlab_dataset/dataset/cifar10'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

net = ResNet18()

net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

epi = 4
checkpoint = torch.load(parameter_path+f'/checkpoint/main_{epi}.pth')
net.load_state_dict(checkpoint['net'])

classes = {'airplane':0, 'automobile':1, 'bird':2, 'cat':3, 'deer':4,
           'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

testset = General_Loader_withpath(is_train=False,  transform=transform_test, name_dict=classes, path=data_path)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

In [45]:
criterion = nn.CrossEntropyLoss()
global best_acc
net.eval()
test_loss = 0
correct = 0
total = 0
result_list = []
with torch.no_grad():
    for batch_idx, (inputs, targets, path) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        _, predicted = outputs.max(1)
        confidence = torch.max(F.softmax(outputs/10, dim=-1))
        result_list.append((float(confidence), float(predicted.eq(targets).sum().item()), path[0]))

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

In [46]:
n = len(result_list)
result_list.sort(key=lambda x:x[0], reverse=True)
result_dict = dict()
for i in range(9):
    result_dict[i] = result_list[i*(n//9):(i+1)*(n//9)]
    # print(len(result_dict[i]))

In [47]:
print(len(result_dict))
for i in range(9):
    count = 0
    for j in range(len(result_dict[i])):
        if result_dict[i][j][1]==0.0:
            count += 1
    print(count)
print(result_dict[2][-1])

9
3
22
35
77
136
261
426
524
709
(0.993591845035553, 1.0, '/home/hinton/NAS_AIlab_dataset/dataset/cifar10/test/automobile/automobile_s_000885.png')
